In [1]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
import matplotlib.patches as patches
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

import tensorflow as tf
import keras as keras

Using TensorFlow backend.
/home/aldopedraza/Documentos/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


In [2]:
path_models = './extra_files/models'

# Load the trained model

In [3]:
def load_model(model_path, height, width):
    # 1: Build the Keras model
    K.clear_session() # Clear previous models from memory.

    if height == 300:
        model = ssd_300(image_size=(height, width, 3),
                        n_classes=20,
                        mode='inference',
                        l2_regularization=0.0005,
                        scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO are [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                        aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5],
                                                 [1.0, 2.0, 0.5]],
                        two_boxes_for_ar1=True,
                        steps=[8, 16, 32, 64, 100, 300],
                        offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                        clip_boxes=False,
                        variances=[0.1, 0.1, 0.2, 0.2],
                        normalize_coords=True,
                        subtract_mean=[123, 117, 104],
                        swap_channels=False,
                        confidence_thresh=0.5,
                        iou_threshold=0.45,
                        top_k=200,
                        nms_max_output_size=400)
    elif height == 512:
        model = ssd_512(image_size=(height, width, 3),
                n_classes=20,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=False,
               confidence_thresh=0.5,
               iou_threshold=0.45,
               top_k=200,
               nms_max_output_size=400)

    # 2: Load the trained weights into the model.
    # TODO: Set the path of the trained weights.
    model.load_weights(model_path, by_name=True)

    # 3: Compile the model so that Keras won't complain the next time you load it.

    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

    model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
    
    return model

In [4]:
def load_graph(frozen_graph_filename):
    '''
    Load a frozen graph
    frozen_grapth_filename: path in the disk
    return: tensorflow graph
    '''
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [5]:
# Load models and weights
model_pascal_voc_07_plus_12_300x300 = '../weights/pascal_voc/pascal_voc_07_plus_12_300x300.h5'
model = load_model(model_pascal_voc_07_plus_12_300x300, 300, 300)

In [6]:
# Save models as .pbtxt or pb
tf.train.write_graph(K.get_session().graph, path_models, "ssd_model.pbtxt", as_text=True)
tf.train.write_graph(K.get_session().graph, path_models, "ssd_model.pb", as_text=False)

'./extra_files/models/ssd_model.pb'

In [7]:
graph = load_graph(path_models + '/ssd_opt_model.pb')

# save data from a grapth to display on tensorboard
# tensorboard --logdir=/home/aldopedraza/Documentos/ssd/extra_files/logs
# the path has to be absolute
with tf.Session(graph=graph) as sess:
    tf.summary.FileWriter('../logs_ssd', sess.graph)

In [ ]:
# for op in graph.get_operations():
#     print(op.name)

In [ ]:
# from tensorboard import reshape as reshape
# count = 0
# for i in model.layers:
#     print(count, i)
#     count +=1

In [8]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
sess = keras.backend.get_session()
save_path = saver.save(sess, path_models + "/ssd_model.ckpt")

## Freeze the Graph

In [9]:
# Freeze the graph
import tensorflow as tf
from tensorflow.python.tools import freeze_graph

input_graph_path = path_models + '/ssd_model.pb'
checkpoint_path = path_models + '/ssd_model.ckpt'
input_saver_def_path = ""
input_binary = True
output_node_names = "predictions/concat"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = path_models + '/ssd_frozen_model.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from ./extra_files/models/ssd_model.ckpt
INFO:tensorflow:Froze 71 variables.
INFO:tensorflow:Converted 71 variables to const ops.


## Optimize for inference

In [10]:
!python3 -m tensorflow.python.tools.optimize_for_inference \
--input=./extra_files/models/ssd_frozen_model.pb \
--output=./extra_files/models/ssd_opt_model.pb \
--frozen_graph=True --input_names=input_1 \
--output_names=predictions/concat

In [11]:
# summary model
# bazel build tensorflow/tools/graph_transforms:summarize_graph && \
bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
--in_graph=../ssd/extra_files/models/ssd_opt_model.pb

SyntaxError: invalid syntax (<ipython-input-11-cd9c694a31f2>, line 2)

# Convert to tflite model

In [12]:
graph_def_file = "/home/aldopedraza/Documentos/ssd/extra_files/models/ssd_opt_model.pb"
input_arrays = ["input_1"]
output_arrays = ["predictions/concat"]

converter = tf.contrib.lite.TocoConverter.from_frozen_graph(
  graph_def_file, input_arrays, output_arrays, input_shapes={"input_1" : [1, 300, 300, 3]})
tflite_model = converter.convert()
open("/home/aldopedraza/Documentos/ssd/extra_files/models/converted_model.tflite", "wb").write(tflite_model)

105440208

In [7]:
# !toco \
#   --graph_def_file=/home/aldopedraza/Documentos/ssd/extra_files/models/ssd_opt_model.pb \
#   --output_file=/home/aldopedraza/Documentos/ssd/extra_files/models/tflite_model.tflite \
#   --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE \
#   --inference_type=QUANTIZED_UINT8 \
#   --input_shape="1,512,512,3" \
#   --input_array=input_1 \
#   --output_array=predictions/concat \
#   --std_dev_values=127.5 --mean_value=127.5

!toco \
--output_file=/home/aldopedraza/Documentos/ssd/extra_files/models/tflite_model.tflite \
--graph_def_file=/home/aldopedraza/Documentos/ssd/extra_files/models/ssd_opt_model.pb \
--output_format=TFLITE \
--inference_type=QUANTIZED_UINT8 \
--inference_input_type=FLOAT \
--input_arrays=input_1 \
--output_arrays=predictions/concat \
--input_shapes=1,512,512,3 \

2018-10-12 17:46:52.854756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-10-12 17:46:52.855189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: GeForce GTX 1050 major: 6 minor: 1 memoryClockRate(GHz): 1.531
pciBusID: 0000:01:00.0
totalMemory: 1.94GiB freeMemory: 1.14GiB
2018-10-12 17:46:52.855210: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-10-12 17:46:53.095451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-10-12 17:46:53.095486: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 
2018-10-12 17:46:53.095502: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 0:   N 
2018-10-12 17:46:53.095636: I tensorflow/core/common_runtime/gpu/gpu_device.cc:110

In [18]:
import tensorflow as tf
graph_def_file = '/home/aldopedraza/Documentos/ssd/extra_files/models'
input_arrays = ["input_1"]
output_arrays = ["predictions/concat"]

converter=tf.contrib.lite.TocoConverter.from_saved_model(graph_def_file, tag_set='serve')
converter.post_training_quantize=True
tflite_quantized_model=converter.convert()
open("/home/aldopedraza/Documentos/ssd/extra_files/models/quat_model.tflite", "wb").write(tflite_quantized_model)

RuntimeError: MetaGraphDef associated with tags serve could not be found in SavedModel. To inspect available tag-sets in the SavedModel, please use the SavedModel CLI: `saved_model_cli`

## Test model to see if it's compatible with tflite

In [23]:
import numpy as np
import tensorflow as tf
from scipy import misc

# Read image
image_test = misc.imread('./image_test.jpg')
image_test = misc.imresize(image_test, size=(300, 300))
image_test = image_test.reshape(1, 300, 300, 3)

# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path="/home/aldopedraza/Documentos/ssd/extra_files/models/converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(image_test, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data[0])

/home/aldopedraza/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel/__main__.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/aldopedraza/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[[9.9316496e-01 2.0371385e-04 2.3979013e-04 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [9.9452364e-01 1.7735115e-04 2.3558663e-04 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [9.9433315e-01 1.8901868e-04 1.7332514e-04 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 ...
 [4.4124798e-05 2.7008391e-05 9.0820151e-10 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [2.3393070e-04 4.1228341e-05 2.5147060e-09 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [5.1122677e-01 3.2095129e-05 1.0950321e-09 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]]


In [51]:
print(output_data[0][1][32])

0.2


In [8]:
output_details[]

[{'name': 'predictions/concat',
  'index': 152,
  'shape': array([   1, 8732,   33], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]